In [16]:
from azureml.core.run import Run

In [15]:
%%writefile invasive_model.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os
import torch.nn.functional as F
import torchvision.models as models
from azureml.core import Run

print(torch.__version__)

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--architecture', type=str)
parser.add_argument('--lr', type=float)
parser.add_argument('--n_neurons', type=int)
parser.add_argument('--epochs',type=int)
parser.add_argument('--data_path',type=str)
args = parser.parse_args()

if args.n_neurons == None: args.n_neurons = 512
if args.epochs == None: args.epochs = 10
if args.architecture == None: args.architecture = 'resnet34'
if args.lr == None: args.lr = 0.01

print(args.data_path)


def create_body(architecture):
    if architecture == 'resnet34':
      model = models.resnet34(pretrained=True)
    elif architecture == 'resnet18':
      model = models.resnet18(pretrained=True)
    "Cut off the body of a typically pretrained `model` at `cut` (int) or cut the model as specified by `cut(model)` (function)."
    return nn.Sequential(*list(model.children())[:-1])


class conv_net(nn.Module):
    def __init__(self,fc_neurons,architecture):
        super().__init__()
        self.loss = []
        self.acc = []

        self.body = create_body(architecture)

        conv_out_size = self._get_conv_out([3,224,224])
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, fc_neurons),
            nn.ReLU(),
            nn.Linear(fc_neurons, 2),
            nn.LogSoftmax()
        )

    def _get_conv_out(self, shape):
        o = self.body(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.body(x).view(x.size()[0], -1)
        return self.fc(conv_out)


    def fit(self, train_loader, valid_loader, lr, epochs):
        criterion = nn.NLLLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        epochs = epochs
        print_every = 50
        for e in range(epochs):
            running_loss = 0
            for step, (x,y) in enumerate(train_loader):
                y = y.to(device)
                x = x.to(device)
                
                optimizer.zero_grad()
                
                # Forward and backward passes
                output = self.forward(x)
                loss = criterion(output, y)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                
            if step > 1:
                if step % print_every == 0: 
                    print("Epoch: {}/{}... ".format(e+1, epochs),
                    "Loss: {:.4f}".format(running_loss/print_every))
                    run.log('Training Loss', running_loss/print_every)
                    running_loss = 0
                
            loss, acc = self.score(valid_loader)
            self.loss.append(loss)
            self.acc.append(acc)
            
    def score(self, valid_loader):
        test_loss = 0
        correct = 0
        preds_c = []
        with torch.no_grad():
          for data, target in valid_loader:
              data = data.to(device)
              target = target.to(device)
              output = self(data)
              test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
              pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
              preds_c.append(pred)
              correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(valid_loader.dataset)

        test_acc = 100. * correct / len(valid_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(valid_loader.dataset),test_acc))
        return test_loss, test_acc

run = Run.get_context()

#data preprocessing and train/test splitting
transforms = transforms.Compose([transforms.Resize([224,224]),transforms.ToTensor()])

image_datasets = datasets.ImageFolder(args.data_path,transform=transforms)
train_set, val_set = torch.utils.data.random_split(image_datasets, [int(len(image_datasets)*0.8), int(len(image_datasets)*0.2)])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False)

print('train length ', len(train_loader), '; validation length ', len(valid_loader))
print('n classes ', image_datasets.classes)

#initialize/instantiate model
model = conv_net(args.n_neurons,args.architecture)

#todo param for freezing base layers
n_layers = len(list(model.parameters()))-6
for layer in model.parameters():
    layer.requires_grad=False
    n_layers -=1
    if n_layers==0:
        break

#loss function
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9)

#use GPU if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)
    
model.to(device)

#fit
model.fit(train_loader, valid_loader, args.lr, args.epochs)

loss, acc = model.score(valid_loader)
run.log('Final Loss', loss)
run.log('Final Accuracy',acc)
run.log_list('Training Loss', model.loss)
run.log_list('Training Accuracy', model.acc)

print(model.loss)
print(model.acc)

os.makedirs('./outputs/model', exist_ok=True)

# todo model to onnx
torch.save(model.state_dict(), './outputs/model/invasive-model.state')


Overwriting invasive_model.py


In [3]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Datastore, Dataset

In [4]:
from azureml.core import Workspace, Experiment
import json
with open('/Users/paulbruffett/python_class/azure_credentials.json') as f:
    credentials = json.load(f)
    
auth = azureml.core.authentication.ServicePrincipalAuthentication(credentials['directory_id'],credentials['app_id'],credentials['app_key'])
ws = Workspace('3bdbda93-8c3a-472b-bdde-25e3028fc307','mlworkspace','pbml',auth=auth)
ws_details = ws.get_details()
print('Name:\t\t{}\nLocation:\t{}'
      .format(ws_details['name'],
              ws_details['location']))

Name:		pbml
Location:	eastus2


In [5]:
from azureml.core import Experiment

script_folder = './invasive_model'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='invasive-species-pytorch')

In [6]:
datastore = Datastore.get(ws, 'workspaceblobstore')

In [7]:
from azureml.core.dataset import Dataset

datastore_paths = [(datastore, 'invasive/invasive/')]
invasive_ds = Dataset.File.from_files(path=datastore_paths)

In [8]:
invasive_ds = invasive_ds.register(workspace = ws,
                           name = 'invasive dataset',
                           description='training and test dataset',
                                  create_new_version=True)

In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-01-25T05:26:35.474000+00:00', 'errors': None, 'creationTime': '2020-01-24T17:32:16.091995+00:00', 'modifiedTime': '2020-01-24T23:25:44.867920+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT600S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [10]:
import shutil

# the training logic is in the keras_mnist.py file.
shutil.copy('./invasive_model.py', script_folder)

'./invasive_model/invasive_model.py'

In [11]:
dataset = Dataset.get_by_name(ws, 'invasive dataset')

# list the files referenced by mnist dataset
dataset.to_path()

['/invasive/100.jpg',
 '/invasive/1000.jpg',
 '/invasive/1002.jpg',
 '/invasive/1003.jpg',
 '/invasive/1004.jpg',
 '/invasive/1005.jpg',
 '/invasive/1008.jpg',
 '/invasive/1009.jpg',
 '/invasive/101.jpg',
 '/invasive/1010.jpg',
 '/invasive/1011.jpg',
 '/invasive/1016.jpg',
 '/invasive/1019.jpg',
 '/invasive/102.jpg',
 '/invasive/1022.jpg',
 '/invasive/1026.jpg',
 '/invasive/1028.jpg',
 '/invasive/103.jpg',
 '/invasive/1030.jpg',
 '/invasive/1033.jpg',
 '/invasive/1035.jpg',
 '/invasive/1036.jpg',
 '/invasive/1039.jpg',
 '/invasive/104.jpg',
 '/invasive/1040.jpg',
 '/invasive/1041.jpg',
 '/invasive/1043.jpg',
 '/invasive/1044.jpg',
 '/invasive/1045.jpg',
 '/invasive/1047.jpg',
 '/invasive/1048.jpg',
 '/invasive/1049.jpg',
 '/invasive/105.jpg',
 '/invasive/1050.jpg',
 '/invasive/1051.jpg',
 '/invasive/1056.jpg',
 '/invasive/1057.jpg',
 '/invasive/1058.jpg',
 '/invasive/106.jpg',
 '/invasive/1060.jpg',
 '/invasive/1061.jpg',
 '/invasive/1062.jpg',
 '/invasive/1063.jpg',
 '/invasive/1064.j

In [12]:
from azureml.train.dnn import PyTorch


script_params = {
    '--data_path': dataset.as_named_input('invasive').as_mount(),
    '--lr': 0.01,
    '--n_neurons': 256,
    '--epochs': 10,
    '--architecture': 'resnet18'
}

est = PyTorch(source_directory=script_folder,
                 script_params=script_params,
                 compute_target=compute_target, 
                 entry_script='invasive_model.py', 
                  use_gpu=True,
                 pip_packages=['torch','torchvision','azureml-dataprep[pandas,fuse]','matplotlib'])

WARNING - framework_version is not specified, defaulting to version 1.3.
WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['torch', 'torchvision']. We cannot guarantee image build will succeed.


In [13]:
run = exp.submit(est)